<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Retrieval Augmented Generation (RAG) Baseline
## Overview  
In this time, we will check the baseline code.
The goal of this project is to provide students with hands-on experience in handling and enhancing Large Language Models (LLMs) provided by [**Upstage**](https://www.upstage.ai) (Solar).

You can use any engineering method for improving benchmark performance excluding direct training (Fine-tuning).

*Collecting data directly related to the test set is considered cheating e.g., using MMLU-pro dataset or EWHA.pdf for KB*

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
# load datasets
from datasets import load_dataset

cache_dir = "/content/drive/MyDrive/Upstage_Project/db"
ds = load_dataset("BoltMonkey/psychology-question-answer", cache_dir=cache_dir)

README.md: 0.00B [00:00, ?B/s]

data/train/train.json:   0%|          | 0.00/74.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/197180 [00:00<?, ? examples/s]

In [10]:
# @title set API key
# First, enroll your API key as the colab key.
from pprint import pprint
import os

import warnings

warnings.filterwarnings("ignore")

from IPython import get_ipython

upstage_api_key_env_name = "upstage_api_key"


def load_env():
    if "google.colab" in str(get_ipython()):
        # Running in Google Colab
        from google.colab import userdata

        upstage_api_key = userdata.get(upstage_api_key_env_name)
        # print(upstage_api_key)
        return os.environ.setdefault(upstage_api_key_env_name, upstage_api_key)
    else:
        # Running in local Jupyter Notebook
        from dotenv import load_dotenv

        load_dotenv()
        return os.environ.get(upstage_api_key_env_name)


UPSTAGE_API_KEY = load_env() # Setting API Key

In [21]:
# set parameters
db_path = "./drive/MyDrive/Upstage_Project/db" # folder path containing ewah.pdf
assets_path = "./drive/MyDrive/Upstage_Project/assets"

# Baseline

In [45]:
!pip3 install -qU openai python-dotenv PyPDF2 langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv tiktoken faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 48.6 MB/s eta 0:00:00


In [14]:
from langchain_upstage import UpstageDocumentParseLoader
import os

layzer = UpstageDocumentParseLoader(api_key=UPSTAGE_API_KEY,file_path=os.path.join(db_path, 'ewha.pdf'), output_format="text")
docs = layzer.load()  # or layzer.lazy_load()

In [40]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

# 2. Split
text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=500, chunk_overlap=100, language=Language.HTML
)

splits = text_splitter.split_documents(docs)
print("Splits:", len(splits))


Splits: 120


In [43]:
print(splits[0].page_content)

이화여자대학교 학칙1946. 8. 15. 제정
2017. 8. 16. 개정제1장 총칙제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과
그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에
공헌할 수 있는 지도여성을 양성함을 목적으로 한다.제2조(명칭) 본교는 이화여자대학교라 부른다.
제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)제2장 편제제4조(대학 및 대학원) ① 본교에는 다음 각 호의 대학을 둔다.1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범
대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각
대학”이라 한다) (개정 2016.6.16.)
2. 호크마(HOKMA)교양대학
② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대
학원, 디자인대학원


In [44]:
print(splits[1].page_content)

라 한다) (개정 2016.6.16.)
2. 호크마(HOKMA)교양대학
② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대
학원, 디자인대학원, 사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보
건융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한다).
(개정 2016.6.16., 2017.5.15.)
[전문개정 2015.11.27.]제5조(학부․학과․전공 및 정원) ① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표
1과 같다. (개정 2015.5.8., 2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.15.)
② 모집단위별 입학정원의 일부는 입학전형에 따라 2개 이상의 모집단위를 통합하여 모집
할 수 있다. (개정 1999.2.9., 2017.5.15.)
③ 제2항에 따라 통합된 모집단위로 입학한 학생과 대학 또는 학부 등 광역화된


In [48]:
%mkdir /content/drive/MyDrive/Upstage_Project/faiss_vectorstore

In [49]:
%cd /content/drive/MyDrive/Upstage_Project/faiss_vectorstore

/content/drive/MyDrive/Upstage_Project/faiss_vectorstore


In [50]:
!pwd

/content/drive/MyDrive/Upstage_Project/faiss_vectorstore


In [51]:
from langchain_upstage import UpstageEmbeddings
from langchain_community.vectorstores import FAISS

upstage_embeddings = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model="solar-embedding-1-large-passage")

# create and save a FAISS vectorstore
vectorstore = FAISS.from_documents(documents=splits,
                                   embedding=upstage_embeddings)
vectorstore.save_local("faiss_index_ewha")

In [19]:
# read samples.csv file

import pandas as pd

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers

In [22]:
prompts, answers = read_data(os.path.join(assets_path, 'testset.csv'))

In [63]:
# from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_upstage import ChatUpstage
from tqdm import tqdm

# load db
ewha_db = FAISS.load_local("./faiss_index_ewha",
                           upstage_embeddings,
                           allow_dangerous_deserialization=True)

# retriever
retriever = ewha_db.as_retriever(search_type="mmr", search_kwargs={'k': 5, 'lambda_mult': 0.15})

# llm
llm = ChatUpstage(api_key = UPSTAGE_API_KEY,
                  model="solar-pro2")

# prompt
prompt_template = ChatPromptTemplate.from_messages([
    ("system", """You are an expert academic advisor specializing in Ewha Womans University regulations and policies. You are smart, brilliant, and good at math.

## YOUR MISSION:
Provide a precise answer to multiple-choice questions (4-10 options) about Ewha Womans University regulations using ONLY the information from the given context.

## INSTRUCTIONS:
1. Analyze the Context: Carefully read and understand a question and all provided documents
2. Evaluate Each Option: Compare each answer choice against the context
3. Show Your Reasoning: Explain step-by-step how you arrived at your answer
4. State Your Final Answer: Clearly indicate your choice in the required format

## CONSTRAINTS:
- Use ONLY information from the provided context
- If the answer cannot be determined from the context, state: "The information is not present in the context."
- Do NOT make assumptions beyond what is explicitly stated

## REQUIRED OUTPUT FORMAT:
After your reasoning, you MUST conclude with your final answer in EXACTLY this format:

[ANSWER]: (X) [brief answer text]

Where X is the option letter (A, B, C, D, etc.)

Example:
[ANSWER]: (B) 2"""),

    ("human", """Question: {question}
    ---
    Context: {context}
    ---
    Please provide your step-by-step reasoning and final answer.""")
])

# combine documents
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG chain
rag_chain = prompt_template | llm

responses = []

for i, prompt in enumerate(prompts[:25]):
    # get docs which are relevant to query using retriever
    docs = retriever.get_relevant_documents(prompt)

    # call RAG chain
    response = rag_chain.invoke({"question": prompt, "context": format_docs(docs)})
    responses.append(response.content)

    if i == 0:
        print(f"📌 question: {prompt}")
        print(f"✅ context: {format_docs(docs)}")

📌 question: QUESTION1) 재학 중인 학생이 휴학을 하려면 학기 개시일로부터 며칠 이내에 휴학을 신청하야하나요?
(A) 30일
(B) 45일 
(C) 60일
(D) 90일
✅ context: 1988.7.28)
③ 1회의 휴학기간은 1년 이내로 한다. 다만, 교과과정상의 필요에 따라 총장이 지정하는
학부, 학과 또는 전공에 있어서는 이를 1년으로 한다. (개정 1996.2.15)
④ 휴학기간은 통산하여 3년(건축학전공의 경우 4년, 의예과의 경우 3학기)을 초과할 수
없다. 다만, 임신, 출산 및 육아로 인한 휴학, 창업으로 인한 휴학은 2년 이내의 기간을,
군복무로 인한 휴학은 의무복무기간을 추가 휴학기간으로 허가할 수 있다. (개정 2013.
2.25., 2015.9.18., 2016.2.16.)
⑤ 삭제 (1985.9.9.)
⑥ 재학 중인 자가 휴학을 하고자 하는 경우 학기개시일로부터 90일 이내에 휴학을 신청하
여야 한다. (신설 2015.9.18.)
⑦ 신입생, 편입학한 학생, 재입학한 학생은 중대한 질병 및 그에 준하는 사유를 제외하고
는 입학 후 첫 학기를 휴학할 수 없다. (신설 2015.9.18.)2 - 2 - 6이화여자대학교 학칙⑧ 제4항에서 정한 휴

였거나 제1항의 입학사정에 사용된 전형자료에 허위나
부정이 있다고 판단되는 경우에는 교무회의의 의결을 거쳐 합격취소 및 입학허가취소를 할
수 있다. (신설 2005.7.6)
③ 제2항의 합격취소 및 입학허가취소의 사유와 절차에 관한 사항은 총장이 따로 정한다.
(신설 2005.7.6.)[제목개정 2015.2.6.]제19조(입학금 등) ① 입학이 허가된 자는 정해진 기일 내에 입학금 기타 납입금을 납입하여
야 한다. (개정 1997.12.23)② 정당한 이유 없이 제1항의 절차를 이행하지 아니한 자에게는 입학허가를 취소한다.제20조(보증인) ① 학생은 부모 기타의 자로서 학생의 재학중 학비 기타 신상에 관계되는 모
든 사항에 대하여 책임을 질만한 능력이 있는 자를 보증인으로 정한다

In [64]:
for idx, (prompt, response) in enumerate(zip(prompts[:25], responses[:25])):
    print(f"{idx+1}.", prompt)
    print(f"     {response}")
    print('-'*10)

1. QUESTION1) 재학 중인 학생이 휴학을 하려면 학기 개시일로부터 며칠 이내에 휴학을 신청하야하나요?
(A) 30일
(B) 45일 
(C) 60일
(D) 90일
     To determine the correct answer, I will carefully analyze the provided context and evaluate each option:

1. **Context Analysis**: The relevant information about the deadline for applying for a leave of absence is found in **⑥ 재학 중인 자가 휴학을 하고자 하는 경우 학기개시일로부터 90일 이내에 휴학을 신청하여야 한다. (신설 2015.9.18.)**  
   - This clause explicitly states that a currently enrolled student must apply for a leave of absence **within 90 days from the start of the semester**.

2. **Option Evaluation**:
   - (A) 30일: Incorrect. The context does not mention 30 days.
   - (B) 45일: Incorrect. The context does not mention 45 days.
   - (C) 60일: Incorrect. The context does not mention 60 days.
   - (D) 90일: Correct. The context explicitly states **90일** (90 days).

3. **Conclusion**: The only option that matches the information in the context is **(D) 90일**.

[ANSWER]: (D) 90일
----------
2. QUESTION2) '재입학은 a회에 한하여 할 수

In [65]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

In [66]:
# print accuracy

cnt = 0

for i, (answer, response) in enumerate(zip(answers[:25], responses[:25])):
    print("-"*10)
    generated_answer = extract_answer(response)
    # print(response)
    # check
    if generated_answer:
        print(f"{i+1}. generated answer: {generated_answer}, answer: {answer}")
    else:
        print(f"{i+1}. extraction fail")


    if generated_answer == None:
        continue
    if generated_answer in answer:
        cnt += 1

print()
print(f"acc: {(cnt/25)*100}%")

----------
1. generated answer: D, answer: (D)
----------
2. generated answer: A, answer: (A)
----------
3. generated answer: C, answer: (C)
----------
4. generated answer: C, answer: (D)
----------
5. generated answer: C, answer: (C)
----------
6. generated answer: C, answer: (B)
----------
7. generated answer: C, answer: (D)
----------
8. generated answer: C, answer: (C)
----------
9. generated answer: C, answer: (C)
----------
10. generated answer: B, answer: (B)
----------
11. generated answer: B, answer: (B)
----------
12. generated answer: B, answer: (B)
----------
13. generated answer: C, answer: (C)
----------
14. generated answer: C, answer: (C)
----------
15. generated answer: A, answer: (A)
----------
16. generated answer: B, answer: (B)
----------
17. generated answer: I, answer: (B)
----------
18. generated answer: D, answer: (D)
----------
19. generated answer: D, answer: (D)
----------
20. generated answer: D, answer: (D)
----------
21. generated answer: B, answer: (B)
-